In [1]:
#Notebook encargada de crear el dataset de lluvias

In [6]:
%config Completer.use_jedi = False

In [1]:
import os
import pandas as pd
from pathlib import Path
import georasters as gr
from osgeo import gdal
import datetime
import numpy as np
from pathlib import Path
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
lista_paths = list()
base = '/opt/datos/gfs/lluvias_00/gfs.0p25.'
ext = '.grib2'
start_date = "2015-01-15 00:00"
finish_date = "2021-08-21 00:00"
idx = pd.date_range(start_date, finish_date, freq='24H').strftime("%Y%m%d%H")
f = ['.f003', '.f006', '.f009', '.f012', '.f015', '.f018', '.f021', '.f024']

for time in idx:
    for k in f:
        time_new = base+time+k+ext
        lista_paths.append(time_new)

In [9]:
print(lista_paths[1500])
print(lista_paths.index('/opt/datos/gfs/lluvias_00/gfs.0p25.2015072100.f015.grib2'))

/opt/datos/gfs/lluvias_00/gfs.0p25.2015072100.f015.grib2
1500


In [54]:
X = np.zeros((19288,137,157))
for i,path in enumerate(lista_paths): #i va de 0 a 19287
    grib = gdal.Open(path)
    band1 = grib.GetRasterBand(1) #prec total
    band2 = grib.GetRasterBand(2) #prec convectiva
    data1 = band1.ReadAsArray()
    data2 = band2.ReadAsArray()
    data = data1 + data2
    
    if(i%2 == 0):
        X[i,:,:] = data
    elif(i%2 == 1):
        grib_anterior = gdal.Open(lista_paths[i-1])
        band1_a = grib_anterior.GetRasterBand(1) #prec total
        band2_a = grib_anterior.GetRasterBand(2) #prec convectiva
        data1_a = band1_a.ReadAsArray()
        data2_a = band2_a.ReadAsArray()
        data_a = data1_a + data2_a
        
        data_final = data - data_a
        X[i,:,:] = data_final

In [55]:
X.shape

(19288, 137, 157)

In [56]:
np.save("/home/joaquinfunes/X.npy", X)

## Validacion 

In [ ]:
X = np.load('/home/joaquinfunes/X.npy')

In [19]:
# f impares (APCP03): f003 - f009 - f015 - f021
#                     X[0] - X[2] - X[4] - X[6]
grib = gdal.Open('/opt/datos/gfs/lluvias_00/gfs.0p25.2015011700.f015.grib2')
band1 = grib.GetRasterBand(1)
band2 = grib.GetRasterBand(2)
data1 = band1.ReadAsArray()
data2 = band2.ReadAsArray()
data = data1 + data2
print(np.array_equal(X[20,:,:], data))

True


In [16]:
# f pares (APCP06): f006 - f012 - f018 - f024
#                   X[1] - X[3] - X[5] - X[7]
#                   X[9] - X[11]  X[13]  X[15]
grib = gdal.Open('/opt/datos/gfs/lluvias_00/gfs.0p25.2015020300.f024.grib2')
grib_a = gdal.Open('/opt/datos/gfs/lluvias_00/gfs.0p25.2015020300.f021.grib2')
band1 = grib.GetRasterBand(1)
band2 = grib.GetRasterBand(2)
data1 = band1.ReadAsArray()
data2 = band2.ReadAsArray()
data = data1 + data2
band1_a = grib_a.GetRasterBand(1)
band2_a = grib_a.GetRasterBand(2)
data1_a = band1_a.ReadAsArray()
data2_a = band2_a.ReadAsArray()
data_a = data1_a + data2_a

data_final = data - data_a
print(np.array_equal(X[159,:,:], data_final))

True


In [ ]:
# (8*index) + 1
#00) 2015-01-15 f006 --> X[1]
#01) 2015-01-16 f006 --> X[9]
#02) 2015-01-17 f006 --> X[17]
#12) 2015-01-27 f006 --> X[97]
#15) 2015-01-30 f006 --> X[121]
#16) 2015-01-31 f006 --> X[129]
#18) 2015-02-02 f006 --> X[145]

# f012 = f006 + 2
# f018 = f006 + 4
# f024 = f006 + 6